In [1]:
import os
import time
print("Running on ", time.asctime())

import numpy             as np 
import scipy.optimize    as so

import scipy.stats

import invisible_cities.core.fit_functions  as fitf

Running on  Thu Oct  5 19:49:48 2017


In [2]:
%load_ext autoreload

%autoreload 2

# Very simple example: fitting a straight line from pes to keV conversion

This example has a bad error behavior, being too small for huge quantities, but it is an actual example I had to deal with

In [3]:
def line(x, m, n):
    return m*x + n

In [4]:
y  = np.array([ 9.108e3, 10.34e3,   1.52387e5,   1.6202e5])
ey = np.array([ 3.17   , 13.5   ,  70        ,  21       ])
x  = np.array([29.7    , 33.8   , 481        , 511       ])

### Previous IC implementation

In [5]:
res, cov     = so.curve_fit(line, x, y, p0=(1,1), sigma=ey)
err          = np.diag(cov)**0.5
chi2, pvalue = scipy.stats.chisquare(y, line(x, *res), len(y)-2)
chi2         = chi2/(len(y)-2)

print('Slope    : {0:1.2f}+-{1:1.2f}'  .format(res[0], err[0]))
print('Intercept: {0:1.0f}+-{1:1.0f}'.format(res[1], err[1]))
print('chi2     : {}'  .format(chi2))
print('P-value  : {:e}'.format(pvalue))

Slope    : 317.69+-0.16
Intercept: -331+-13
chi2     : 0.2442279235092592
P-value  : 4.846169e-01


### Proposal

In [6]:
f = fitf.fit(line, x, y, seed=(1,1), sigma=ey)
print('Slope    : {0:1.3f}+-{1:1.3f}'  .format(f.values[0], f.errors[0]))
print('Intercept: {0:1.1f}+-{1:1.1f}'.format(f.values[1], f.errors[1]))
print('chi2     : {}'  .format(f.chi2))
print('P-value  : {:e}'.format(f.pvalue))

Slope    : 317.695+-0.043
Intercept: -331.2+-3.5
chi2     : 13.918715148470396
P-value  : 9.019424e-07


### Comparison with other fitting tools

| Tool             |       Slope       | Intercept         | $\chi^2$     |
| :---------------:|:---------------:  |:-----------------:| :-----:  |
| previous IC      | $317.67\pm0.18$   | $-331\pm13$       | $0.2442$ |
| proposed IC      | $317.670\pm0.043$ | $-331.2\pm3.5$    | $13.919$ |
| ROOT             | $317.693\pm0.031$ | $-331.1\pm3.5$    | $14.069$ |
| Qtiplot (~Origin)| $317.694\pm0.042$ | $-331.2\pm3.5$    | $13.924$ |

Both errors and $\chi^2$ for the new procedure seem to be in agreement with other tools, but old method does not agree.


# Gaussian fit

## Poisson errors

This example shows that, in a poisson smeared scenario, both procedure should yield compatible and reasonable $\chi^2$ 

In [7]:
A     =  1e6
mu    = 1500
sigma =  210
x     = np.linspace(1e3, 2e3, 100)
y     = fitf.gauss(x, A, mu, sigma)
y     = np.random.poisson(y)

### Previous IC implementation

In [8]:
res, cov = so.curve_fit(fitf.gauss, x, y, p0=(A, mu, sigma), sigma=y**0.5)
err      = np.diag(cov)**0.5
chi2, pvalue = scipy.stats.chisquare(y, fitf.gauss(x, *res), len(y)-3)
chi2         = chi2/(len(y)-3)
print('Slope    : {0:1.4e}+-{1:1.1e}'  .format(res[0], err[0]))
print('Intercept: {0:1.2f}+-{1:1.2f}'.format(res[1], err[1]))
print('chi2     : {}'  .format(chi2))
print('P-value  : {:e}'.format(pvalue))

Slope    : 9.9750e+05+-3.2e+03
Intercept: 1501.02+-0.71
chi2     : 0.9911404817192085
P-value  : 1.328398e-21


### Proposal

In [9]:
f = fitf.fit(fitf.gauss, x, y, seed=(A, mu, sigma), sigma=y**0.5)
print('Slope    : {0:1.4e}+-{1:1.1e}'  .format(f.values[0], f.errors[0]))
print('Intercept: {0:1.2f}+-{1:1.2f}'.format(f.values[1], f.errors[1]))
print('chi2     : {}'  .format(f.chi2))
print('P-value  : {:e}'.format(f.pvalue))

Slope    : 9.9750e+05+-3.2e+03
Intercept: 1501.02+-0.71
chi2     : 1.0002564260047784
P-value  : 4.801919e-01


## Non-Poisson errors

This example shows that, in a non-poisson smeared scenario, new method yields reasonable $\chi^2$ but old one does not 

In [10]:
A     =  1e6
mu    = 1500
sigma =  210
x     = np.linspace(1e3, 2e3, 100)
y     = fitf.gauss(x, A, mu, sigma)
y     = np.random.normal(y, np.log(y))

### Previous IC implementation

In [11]:
res, cov = so.curve_fit(fitf.gauss, x, y, p0=(A, mu, sigma), sigma=np.log(y))
err      = np.diag(cov)**0.5
chi2, pvalue = scipy.stats.chisquare(y, fitf.gauss(x, *res), len(y)-3)
chi2         = chi2/(len(y)-3)
print('Slope    : {0:1.4e}+-{1:1.1e}'  .format(res[0], err[0]))
print('Intercept: {0:1.2f}+-{1:1.2f}'.format(res[1], err[1]))
print('chi2     : {}'  .format(chi2))
print('P-value  : {:e}'.format(pvalue))

Slope    : 9.9927e+05+-6.8e+02
Intercept: 1500.06+-0.16
chi2     : 0.06280180199492406
P-value  : 4.755409e-02


### Proposal

In [12]:
f = fitf.fit(fitf.gauss, x, y, seed=(A, mu, sigma), sigma=np.log(y))
print('Slope    : {0:1.4e}+-{1:1.1e}'  .format(f.values[0], f.errors[0]))
print('Intercept: {0:1.2f}+-{1:1.2f}'.format(f.values[1], f.errors[1]))
print('chi2     : {}'  .format(f.chi2))
print('P-value  : {:e}'.format(f.pvalue))

Slope    : 9.9927e+05+-7.1e+02
Intercept: 1500.06+-0.17
chi2     : 0.8927456397739935
P-value  : 7.664243e-01
